# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
# import statements
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier

In [3]:
# load data from database
engine = create_engine('sqlite:///ETL_pipeline_preparation.db')
df = pd.read_sql_table('ETL_preparationTable', engine)
X = df['message']
Y = df[df.columns[4:]]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens
    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [6]:
pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('count_vectorizer', CountVectorizer(tokenizer=tokenize)),
                ('tfidf_transformer', TfidfTransformer())
            ])),

        ])),

        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [7]:
def display_results(y_test, y_pred):
    accuracy = (y_pred == y_test).mean()
    print("Accuracy:", accuracy)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [9]:
#Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# predict on test data
y_pred = pipeline.predict(X_test)

# display results
display_results(y_test, y_pred)

Accuracy: related                   0.758456
request                   0.819957
offer                     0.994465
aid_related               0.582411
medical_help              0.922663
medical_products          0.946956
search_and_rescue         0.973093
security                  0.981396
military                  0.966636
water                     0.936501
food                      0.880996
shelter                   0.915283
clothing                  0.983856
money                     0.974785
missing_people            0.990621
refugees                  0.968327
death                     0.955566
other_aid                 0.864852
infrastructure_related    0.936347
transport                 0.955412
buildings                 0.951568
electricity               0.978475
tools                     0.992159
hospitals                 0.989084
shops                     0.995849
aid_centers               0.987085
other_infrastructure      0.956950
weather_related           0.738007
floods    

In [11]:
print(classification_report(y_test, y_pred, target_names=Y.columns.values))

                        precision    recall  f1-score   support

               related       0.77      0.98      0.86      4970
               request       0.45      0.15      0.22      1133
                 offer       0.00      0.00      0.00        29
           aid_related       0.49      0.18      0.26      2696
          medical_help       0.00      0.00      0.00       499
      medical_products       0.14      0.00      0.01       340
     search_and_rescue       0.00      0.00      0.00       172
              security       0.33      0.01      0.02       120
              military       0.00      0.00      0.00       208
                 water       0.00      0.00      0.00       413
                  food       0.26      0.01      0.02       757
               shelter       0.00      0.00      0.00       543
              clothing       0.00      0.00      0.00       101
                 money       0.50      0.01      0.01       164
        missing_people       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters= {'classifier__estimator__learning_rate': [0.01, 0.02, 0.05],
              'classifier__estimator__n_estimators': [5]}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=2,scoring='f1_micro', n_jobs=-1)

In [13]:
cv.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('count_vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'classifier__estimator__learning_rate': [0.01, 0.02, 0.05], 'classifier__estimator__n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv.fit(X_train, y_train)

# predict on test data
y_pred_final = cv.predict(X_test)

# display results
display_results(y_test, y_pred_final)

Accuracy: related                   0.764145
request                   0.825800
offer                     0.995387
aid_related               0.585486
medical_help              0.923278
medical_products          0.947571
search_and_rescue         0.973555
security                  0.981089
military                  0.968020
water                     0.936193
food                      0.883610
shelter                   0.916513
clothing                  0.984010
money                     0.974170
missing_people            0.990621
refugees                  0.969096
death                     0.956181
other_aid                 0.867005
infrastructure_related    0.936501
transport                 0.955412
buildings                 0.952030
electricity               0.978629
tools                     0.992620
hospitals                 0.987854
shops                     0.996156
aid_centers               0.987392
other_infrastructure      0.957257
weather_related           0.735086
floods    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    Starting Verb Extractor class
    
    This class extract the starting verb of a sentence,
    creating a new feature for the ML classifier
    """

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    # Given it is a tranformer we can return the self 
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [16]:
pipeline2 = Pipeline([
        ('features', FeatureUnion([
            
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),
    
        ('clf', RandomForestClassifier())
    ])

In [17]:
pipeline2.fit(X_train, y_train)

# predict on test data
y_pred = pipeline2.predict(X_test)

# display results
display_results(y_test, y_pred)

print(classification_report(y_test, y_pred, target_names=Y.columns.values))

Accuracy: related                   0.712023
request                   0.817958
offer                     0.995387
aid_related               0.571494
medical_help              0.922355
medical_products          0.947571
search_and_rescue         0.973401
security                  0.981396
military                  0.967866
water                     0.936039
food                      0.882995
shelter                   0.916359
clothing                  0.984317
money                     0.974785
missing_people            0.990775
refugees                  0.968942
death                     0.955105
other_aid                 0.864391
infrastructure_related    0.936501
transport                 0.954490
buildings                 0.952030
electricity               0.979090
tools                     0.992620
hospitals                 0.989391
shops                     0.996156
aid_centers               0.987854
other_infrastructure      0.957411
weather_related           0.731857
floods    

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [18]:
import pickle

pickle.dump(cv, open('MLclassifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.